In [6]:
import featurization
import helpers
import model_bayesian
import model_neighbors

import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

In [5]:
geodf = featurization.all_featurization_keep_recharges("all-sc-bike-data-1101.csv")

Dropping 1 entries where end time was before start time


In [13]:
one_bike = geodf.iloc[1]
neighbors = model_neighbors.get_nearest_neighbor_bikes(one_bike, geodf, 1000)

In [53]:
neighbor_df = geodf.loc[neighbors]

In [62]:
interarrival_times = []
for date in neighbor_df['local_time_start'].dt.date.unique():
    df = neighbor_df[neighbor_df['local_time_start'].dt.date == date]
    arrival_times = sorted(df['local_time_start'].dt.minute)
    for x in range(len(arrival_times)-1):
        interarrival_times.append(arrival_times[x+1]-arrival_times[x])
mean_interarrival_time = np.array(interarrival_times).mean()